# Tutorial: import and export

In this tutorial we demonstrate how to input and output a computational domain in PuMA

In [ ]:
# Run this line only the first time you open a tutorial in Google Colab
!pip install 'git+https://github.com/nasa/puma.git'

First, we must import puma:

In [8]:
import numpy as np
import os
import sys
import pumapy as puma

## Explanation of data structures
First, we give a brief explanation of the PuMA data structures

Microstructures in PuMA are stored in a data structure called a "workspace". The workspace class includes all of the data about the material microstructure, including a 3D matrix of grayscale values, which can be either raw data from tomographic images, or segmented values. Examples of unsegmented and segmented workspaces are demonstrated below: 

In [ ]:
ws_unsegmented = puma.import_3Dtiff("../tests/testdata/200_FiberForm.tif", 1.3e-6)
ws_segmented = ws_unsegmented.copy()
ws_segmented.binarize(90)

%matplotlib widget
plot = puma.plot_slices(ws_unsegmented,'z')
plot = puma.plot_slices(ws_segmented,'z')


the PuMA workspace class also contains other important information about the workspace. This includes the voxel length, which is the physical length, in meters, of each voxel of the image. Typical tomography data ranges from 1e-4 to 1e-7 meters in voxel length. If no value is provided to PuMA, the workspace class defaults to 1e-6 meters as a voxel length. This value can either be set during import-output or it can be set directly for the workspace class. 

An example is shown below, where the voxel length of a workspace class is manually changed

In [ ]:
ws_unsegmented.voxel_length = 1.5e-6

The workspace class can also store the material orientation for each voxel in the domain. This data can either be imported, or can be computed using puma.compute_orientation_st, which applies the structure-tensor method to estimate the material orientation. 

## Workspace import and export
Now we will give examples of input and output for the workspace class. 

PuMA import and export uses three different file types: 3D tiff, vti, and binary (extension is .pumapy)

The 3D tiff images only include the 3d matrix data for the grayscale values. vti and binary, however, includes all data associated with the puma workspace class, including the matrix data, the orientation data (if used) and the voxel length. 

3D tiff images are often the exported data format for tomography imaging, so they are included in PuMA. If you have a tomographic image that uses a different file format, it is recommended to use an external image processing software (FIJI is recommended - https://imagej.net/software/fiji/) and convert the image to a 3D tiff before importing into pumapy.

Below we will show an example of importing, visualizing, and exporting each of the three different file formats:



In [ ]:
ws_vtk = puma.import_vti("../tests/testdata/fibers_with_orientation.vti")

puma.plot_slices(ws_vtk, index=10)
puma.render_contour(ws_vtk,(128,255))
puma.render_orientation(ws_vtk)

print("Voxel Length: ", ws_vtk.voxel_length)

puma.export_vti("../tests/testdata/fibers_with_orientation.vti", ws_vtk)

# Note - there is a known bug with visualizations on MacOS where the visualization is unresponsive when first opened.
# This is an issue with the VTK conda installation. To solve the problem, simply shrink then re-open the window, and
# the visualization should work properly

In [ ]:
ws_binary = puma.import_bin("../tests/testdata/fibers_with_orientation.pumapy")

puma.plot_slices(ws_binary, index=10)
puma.render_contour(ws_binary,(128,255))
puma.render_orientation(ws_binary)

print("Voxel Length: ", ws_binary.voxel_length)

puma.export_bin("../tests/testdata/fibers_with_orientation.vti", ws_binary)


In [ ]:
ws_tiff = puma.import_3Dtiff("../tests/testdata/fibers.tiff")

puma.plot_slices(ws_tiff, index=10)
puma.render_contour(ws_tiff,(128,255))

print("Voxel Length: ", ws_tiff.voxel_length)

puma.export_3Dtiff("../tests/testdata/fibers.tiff", ws_tiff)

As you can see, with the 3D tiff import, the voxel length of the original workspace and the orientation is not preserved. The voxel length can be set by either adding it as an option to the 3D tiff import call, or by setting it directly: 

In [ ]:
ws_tiff = puma.import_3Dtiff("../tests/testdata/fibers.tiff",1.3e-6)
print("Voxel Length - passed to input function: ", ws_tiff.voxel_length)

ws_tiff = puma.import_3Dtiff("../tests/testdata/fibers.tiff")
print("Voxel Length - no input set: ", ws_tiff.voxel_length)
ws_tiff.voxel_length = 1.3e-6
print("Voxel Length - manually changed: ", ws_tiff.voxel_length)
